In [1]:
from diskcache_class import db
from lda_mallet import *
from pre_processing import *
from os import walk
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /home/amh418/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/amh418/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/amh418/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/amh418/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#we are going to run this on 4 different dataset 1. EDML 2. AP 3. Newsgroup 4.Covid-Tweet


############################## reading data for ED corpus
# datafolder = './data/ed_recovery_formatted/Excel'
# #datafolder = 'ed_recovery_topicmodel'
# df = pd.DataFrame(columns=['url','type','photo','date','tags','notes','text','photo_url','reblogged','blog_name'])

# for dirpath,dirnames,filenames in walk(datafolder):
#   for filename in filenames:
#     if filename.endswith('.xlsx'):
#       t = pd.read_excel(datafolder+'/'+filename,names=['url','type','photo','date','tags','notes','text','photo_url','reblogged'])
#       blog_name = t.iloc[0,0].split(':')[1]
#       t['blog_name'] = blog_name
#       df = df.append(t.iloc[3:,:],ignore_index=True)
#       print('blog:{0}   with posts:{1}    and reblogs:{2}  '.format(filename,len(t),len(t[t.reblogged=='yes'])))


# print('number of blogs: {0} - number of posts: {1}'.format(len(set(df.blog_name)),len(df)))
# print('out of {0} documents, {1} are reblogged.'.format(len(df),len(df[df.reblogged == 'yes'])))

# #finding reblogged texts
# texts = sorted(df.text) #sort them to keep smallest post (perhaps original one) at first
# re_texts = []

# while len(texts):
#   t = [texts.pop(0)]#pop first text and find it!
#   if t[0] == ' ' or len(t[0].split())<3: 
#     continue #almost nothing to look
#   i = 0
#   while i<len(texts):
#     if t[0] in texts[i]:
#       t.append(texts.pop(i))
#     else:
#       i += 1
#   if len(t) > 1:
#     re_texts.append(t)


# print('number of unique reblogged texts: {0}'.format(len(re_texts)))
# print('number of unique string in all texts: {0}'.format(len(set(df.text))))

# extra_stopwords = ['isnt','want','cant','wanna','im','could','ive','would','dont','get','also','us','thats','got','ur','wanted',
#                    'may', 'the', 'just', 'can', 'think', 'damn', 'still', 'guys', 'literally', 'hopefully', 'much', 'even', 'rly', 'guess', 'anon']#anything with a length of one
                   

# '''pre-processing'''
# # original_doc_set = list(df.text[df.photo=='no'])
# sel_df = df[df.photo=='no'] #extracting only-text posts
# doc_list = list(sel_df.text)


# ##############reading AP corpus
# text_df = ap_corpus('./data/ap.txt')
# doc_list = list(text_df.text_cleaned)

##############reading Newsgroup corpus
# text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
# doc_list = list(text_df.text_cleaned)

##############Covid Tweet corpus
# doc_list=[]
# with open('./data/covid_tweets','r',encoding='utf-8') as txtfile:
#     doc_list = txtfile.readlines()

##############new ED
import pandas as pd
doc_list = pd.read_csv('./data/ed_new.csv').text


#loading ref corpus for coherene score for lda_mallet
# wiki_docs = loading_wiki_docs('./data/wiki_sampled_5p.txt')
# # wiki_docs = loading_wiki_docs('g:/wiki_corpus_gensim/wiki_full')
# #doing pre-processing on wiki-pedia documents
# pre_processed_wiki, _ = preprocess_data(wiki_docs)
# wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
# del wiki_docs

#loading pre-processed wiki docs and associated vocabulary
import pickle
with open('../wiki_corpus/wiki_full_pre_processed_docs.list','rb') as wfile:
    pre_processed_wiki = pickle.load(wfile)
    
with open('../wiki_corpus/wiki_full_vocab.obj','rb') as wfile:
    wiki_vocab_dict = pickle.load(wfile)

In [19]:
# from bertopic import BERTopic
# from sklearn.datasets import fetch_20newsgroups

# import pandas as pd
# docs = pd.read_csv('./data/ed_new.csv').original_text

# topic_model = BERTopic()
# topics, probs = topic_model.fit_transform(docs)

In [22]:
with open('./data/ed_new.txt','w') as tfile:
    for s in docs:
        tfile.write(s.strip()+'\n')

In [3]:
#removing terms that are not in Wikipedia ref-corpus
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords={})
#generate vocabulary and texts
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

#finding stopwords that are not in Wikipedia and removing those
extra_stopwords = set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys()))
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords=extra_stopwords)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

In [ ]:
#running many many LDA and storing their pair of terms

start = 90; limit=100; step = 10
runs = 3
term_pairs = set()

for num_topics in tqdm(range(start, limit+1, step)):
    model_t = []
    purity_t = []
    coherence_t = []
    contrast_t = []
    for r in range(runs):
        #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=vocab_dict,optimize_interval = 25,workers=1)

        #storing top_terms
        for tn in range(num_topics): 
            tt = model.show_topic(tn,topn=20)

            #saving top_terms and their counts
            top_terms = [i[0] for i in tt]

            #making pair terms
            for i in range(len(top_terms)):
                for j in range(i+1,len(top_terms)):
                    term_pairs.add((top_terms[i],top_terms[j]))
    

In [12]:
#converting set to list
term_pairs_ls = [[i[0],i[1]] for i in list(term_pairs)]

In [4]:
import pickle
# with open('term_pairs.obk','wb') as ofile:
#     pickle.dump(term_pairs_ls,ofile)

import pickle
with open('./data/term_pairs.obk','rb') as ofile:
    term_pairs_ls = pickle.load(ofile)

In [8]:
#computing Coherence for all term pairs
cscore = CoherenceModel(topics=term_pairs_ls,dictionary=wiki_vocab_dict,texts=pre_processed_wiki,coherence='c_npmi',processes=3,topn=2).get_coherence_per_topic()


In [12]:
#Loading the DB
npmi_db = db('./data/wiki_full/')


Load NPMI coherence DB. 
Number of keys : 1934773


In [11]:
#writing into the DB
for i in tqdm(range(len(term_pairs_ls))):
    #if the key does not exist, insert it
    try:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])]
    except:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])] = cscore[i]
    #do the other combination
    try:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])]
    except:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])] = cscore[i]

100%|██████████| 158968/158968 [9:50:37<00:00,  4.49it/s]   


In [28]:
# for k in tqdm(npmi_db.db.iterkeys()):
#     try:
#         npmi_db.db[(k[1],k[0])]
#     except:
#         npmi_db.db[(k[1],k[0])] =  npmi_db.db[k]

125087it [00:23, 5268.58it/s]
